In [1]:
from torch.utils.data import DataLoader

from classifier.file_reader import read_files_from_folder
from classifier.dataset import BertPandasDataset, collate_fn, create_bert_datasets, preprocess_dataframe
from classifier.model import ContinualMultilabelBERTClassifier


In [2]:
SEED = 42
DATASET = "boolq"
MODEL_NAME = "answerdotai/ModernBERT-base"
MINIBATCH_SIZE = 64
N_EPOCHS = 50
TEST_VAL_SET_SIZE = 0.15


df = read_files_from_folder("data/inference_outputs/boolq", file_ext=".csv")
display(df.head())

,input_text,ground_truth,doc_id,small_numeric,small_string,small_bool,medium_numeric,medium_string,medium_bool
0,does ethanol take more energy make that produces,no,0,-2.581821,yes,False,-4.826312,yes,False
1,does ethanol take more energy make that produces,yes,0,-1.581821,yes,True,-3.138812,yes,True
2,is house tax and property tax are same,no,1,-3.108254,no,True,-2.988763,no,True
3,is house tax and property tax are same,yes,1,-6.545754,no,False,-3.426263,no,False
4,is pain experienced in a missing body part or ...,no,2,-4.238685,phantom,False,-4.971416,yes,False


In [3]:
display(len(df["input_text"]))
# Filter the "golden dataset"
display(df["small_bool"].unique())
display(df["medium_bool"].unique())

# First, handle any potential NaN values in the boolean columns
df['small_bool'] = df['small_bool'].fillna(False)
df['medium_bool'] = df['medium_bool'].fillna(False)

# Convert string 'True'/'False' to boolean values
df['small_bool'] = df['small_bool'].astype(str).map({'True': True, 'true': True, 'False': False, 'false': False})
df['medium_bool'] = df['medium_bool'].astype(str).map({'True': True, 'true': True, 'False': False, 'false': False})

# Fill any remaining NaN values (that weren't matched in the mapping) with False
df['small_bool'] = df['small_bool'].fillna(False)
df['medium_bool'] = df['medium_bool'].fillna(False)

# Create a score column using the boolean values directly
df['bool_score'] = df['small_bool'].astype(int) + df['medium_bool'].astype(int)

# Sort by input_text and then by bool_score in descending order
df_sorted = df.sort_values(['input_text', 'bool_score'], ascending=[True, False])

# Keep the first occurrence of each input_text (which will be the one with highest bool_score)
df_filtered = df_sorted.drop_duplicates(subset=['input_text'])

# Drop the temporary score column if you don't need it
df_filtered = df_filtered.drop(columns=['bool_score'])

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

display(df_filtered["small_bool"].unique())
display(df_filtered["medium_bool"].unique())

display(len(df_filtered))

201600

array([False,  True])

array([False,  True])

array([ True, False])

array([ True, False])

3150

In [4]:
text_col = "input_text"
label_cols = ["small_bool", "medium_bool"]

dataset = df_filtered[["input_text", "small_bool", "medium_bool"]]
dataset = preprocess_dataframe(dataset, label_cols=label_cols)

# Create train and validation datasets
train_dataset, val_dataset, tokenizer = create_bert_datasets(
    dataset,
    text_col,
    label_cols,
    model_name=MODEL_NAME,
    max_length=1024,
    val_ratio=0.05,
)

# Create DataLoaders with the custom collate function
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    collate_fn=collate_fn
)

display(f"Training dataset size: {len(train_dataset)}")
display(f"Validation dataset size: {len(val_dataset)}")

'Training dataset size: 2993'

'Validation dataset size: 157'

## Full model training
Training the full model yields strong results but shows overfitting behavior very quickly.
We also exhibit local batch instabilities (observable from loss spikes).
I tried to adjust the classifier architecture to account for those instabilities.
We might need some form of regularization to treat the losses.

In [5]:
# classifier = MultilabelBERTClassifier(
#     model_name=MODEL_NAME,  # Replace with your preferred BERT variant
#     num_labels=len(label_cols),
#     learning_rate=2e-7,
#     weight_decay=0.01,
#     batch_size=1,
#     max_length=128,
#     warmup_ratio=0.1,
#     threshold=0.5,
#     freeze_bert_layers=True
# )
#
# with wandb.init(
#     project="mess-plus-classifier-evaluations",
#     name="minibatch_size-1"
# ):
#
#     # Train the model
#     classifier.fit(train_dataset, val_dataset, epochs=5, early_stopping_patience=2)
#
# wandb.finish()


## Continuous learning approach

In [6]:
cont_model = ContinualMultilabelBERTClassifier(
    model_name=MODEL_NAME,  # Replace with your preferred BERT variant
    num_labels=len(label_cols),
    learning_rate=8e-7,
    weight_decay=0.01,
    batch_size=16,
    max_length=128,
    warmup_ratio=0.1,
    threshold=0.5,
    freeze_bert_layers=True,
    memory_size=0
)


for idx in range(len(dataset)):
    print(f"Fetching sample {idx}/{len(dataset)}...")
    sample = BertPandasDataset(df.loc[idx], text_col, label_cols, tokenizer, 128)
    cont_model.incremental_fit(
        new_train_dataset=sample,
        new_val_dataset=val_dataset,
    )

    if idx % 50 == 0 and idx != 0:
        display(f"Done.")
        break


INFO:classifier.model:Using device: cuda
INFO:classifier.model:Initializing custom BERTClassifier: answerdotai/ModernBERT-base with 2 labels


Fetching sample 0/3150...


INFO:classifier.model:Training on new data only: 10 examples
Evaluating: 100%|██████████| 10/10 [00:01<00:00,  5.19it/s]
INFO:classifier.model:Epoch 1/3 - Time: 2.71s
INFO:classifier.model:  Train Loss: 0.9182
INFO:classifier.model:  Val Loss: 0.5935, Val F1: 0.9170
INFO:classifier.model:  Precision: 0.8521, Recall: 0.9925
Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.07it/s]
INFO:classifier.model:Epoch 2/3 - Time: 1.43s
INFO:classifier.model:  Train Loss: 0.8843
INFO:classifier.model:  Val Loss: 0.5939, Val F1: 0.9170
INFO:classifier.model:  Precision: 0.8521, Recall: 0.9925
Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.07it/s]
INFO:classifier.model:Epoch 3/3 - Time: 1.43s
INFO:classifier.model:  Train Loss: 0.9083
INFO:classifier.model:  Val Loss: 0.5941, Val F1: 0.9170
INFO:classifier.model:  Precision: 0.8521, Recall: 0.9925
INFO:classifier.model:Training on new data only: 10 examples


Fetching sample 1/3150...


Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.10it/s]
INFO:classifier.model:Epoch 1/3 - Time: 1.43s
INFO:classifier.model:  Train Loss: 0.5270
INFO:classifier.model:  Val Loss: 0.5943, Val F1: 0.9170
INFO:classifier.model:  Precision: 0.8521, Recall: 0.9925
Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.09it/s]
INFO:classifier.model:Epoch 2/3 - Time: 1.43s
INFO:classifier.model:  Train Loss: 0.5420
INFO:classifier.model:  Val Loss: 0.5943, Val F1: 0.9170
INFO:classifier.model:  Precision: 0.8521, Recall: 0.9925
Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.09it/s]
INFO:classifier.model:Epoch 3/3 - Time: 1.43s
INFO:classifier.model:  Train Loss: 0.5205
INFO:classifier.model:  Val Loss: 0.5943, Val F1: 0.9170
INFO:classifier.model:  Precision: 0.8521, Recall: 0.9925
INFO:classifier.model:Training on new data only: 10 examples


...

Fetching sample 49/3150...


Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.02it/s]
INFO:classifier.model:Epoch 1/3 - Time: 1.44s
INFO:classifier.model:  Train Loss: 0.6891
INFO:classifier.model:  Val Loss: 0.5980, Val F1: 0.9170
INFO:classifier.model:  Precision: 0.8521, Recall: 0.9925
Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.01it/s]
INFO:classifier.model:Epoch 2/3 - Time: 1.45s
INFO:classifier.model:  Train Loss: 0.6756
INFO:classifier.model:  Val Loss: 0.5980, Val F1: 0.9170
INFO:classifier.model:  Precision: 0.8521, Recall: 0.9925
Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.01it/s]
INFO:classifier.model:Epoch 3/3 - Time: 1.44s
INFO:classifier.model:  Train Loss: 0.6848
INFO:classifier.model:  Val Loss: 0.5980, Val F1: 0.9170
INFO:classifier.model:  Precision: 0.8521, Recall: 0.9925
INFO:classifier.model:Training on new data only: 10 examples


Fetching sample 50/3150...


Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.06it/s]
INFO:classifier.model:Epoch 1/3 - Time: 1.44s
INFO:classifier.model:  Train Loss: 0.7554
INFO:classifier.model:  Val Loss: 0.5981, Val F1: 0.9170
INFO:classifier.model:  Precision: 0.8521, Recall: 0.9925
Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.06it/s]
INFO:classifier.model:Epoch 2/3 - Time: 1.43s
INFO:classifier.model:  Train Loss: 0.8043
INFO:classifier.model:  Val Loss: 0.5982, Val F1: 0.9170
INFO:classifier.model:  Precision: 0.8521, Recall: 0.9925
Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.04it/s]
INFO:classifier.model:Epoch 3/3 - Time: 1.44s
INFO:classifier.model:  Train Loss: 0.8147
INFO:classifier.model:  Val Loss: 0.5982, Val F1: 0.9170
INFO:classifier.model:  Precision: 0.8521, Recall: 0.9925


'Done.'